<a href="https://colab.research.google.com/github/GajulapalliNagaVyshnavi/DockFilterHub/blob/main/Docking_filtering_criteria_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Include complexes with ligand-protein, ligand-small molecule, and ligand-metal ion distances ≥ 0.2 Å.

In [ ]:
!pip install rdkit
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.9 MB/s eta 0:00:00


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

def parse_pdb(file_path):
    """Parse a PDB file and extract atom coordinates and types."""
    atoms = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                atom_type = line[76:78].strip()
                x, y, z = float(line[30:38]), float(line[38:46]), float(line[46:54])
                atoms.append((atom_type, np.array([x, y, z])))
    return atoms

def parse_sdf(file_path):
    """Parse an SDF file and extract atom coordinates."""
    suppl = Chem.SDMolSupplier(file_path, removeHs=False)
    if not suppl or len(suppl) == 0:
        return []

    mol = suppl[0]  # Take the first molecule
    conf = mol.GetConformer()
    atoms = [(atom.GetSymbol(), np.array(conf.GetAtomPosition(atom.GetIdx()))) for atom in mol.GetAtoms()]
    return atoms

def compute_distances(ligand_atoms, target_atoms):
    """Compute minimum distance between ligand atoms and target atoms."""
    min_distance = float('inf')
    for _, lig_coord in ligand_atoms:
        for _, target_coord in target_atoms:
            distance = np.linalg.norm(lig_coord - target_coord)
            min_distance = min(min_distance, distance)
    return min_distance

def check_ligand_complex(ligand_file, protein_file, pocket_file):
    """Check if ligand forms a valid complex with ligand-protein, ligand-small molecule, and ligand-metal ion distances ≥ 0.2 Å."""
    ligand_atoms = parse_sdf(ligand_file)
    protein_atoms = parse_pdb(protein_file)
    pocket_atoms = parse_pdb(pocket_file)

    # Identify metal ions from the pocket or protein
    metal_atoms = [atom for atom in protein_atoms + pocket_atoms if atom[0] in {"ZN", "FE", "MG", "CA", "NA"}]

    # Compute minimum distances
    ligand_protein_dist = compute_distances(ligand_atoms, protein_atoms)
    ligand_small_molecule_dist = compute_distances(ligand_atoms, pocket_atoms)
    ligand_metal_dist = compute_distances(ligand_atoms, metal_atoms)

    # Check if all distances are ≥ 0.2 Å
    return ligand_protein_dist >= 0.2 and ligand_small_molecule_dist >= 0.2 and ligand_metal_dist >= 0.2

# Example usage
ligand_file = "/content/1a0q_ligand.sdf"
protein_file = "/content/1a0q_protein.pdb"
pocket_file = "/content/1a0q_pocket.pdb"

result = check_ligand_complex(ligand_file, protein_file, pocket_file)
print("Complex Valid:", result)


[04:39:48] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


Complex Valid: True


# Include ligands with correct connectivity, element assignment, hybridization, aromaticity, and Kekulé structure

In [ ]:
from rdkit import Chem

def validate_ligand_structure(sdf_file):
    """
    Validates ligand structure by checking connectivity, element assignment,
    hybridization, aromaticity, and Kekulé representation.

    Parameters:
        sdf_file (str): Path to the SDF file containing ligand structure.

    Returns:
        bool: True if the ligand passes all checks, False otherwise.
    """
    supplier = Chem.SDMolSupplier(sdf_file, removeHs=False)
    if not supplier or len(supplier) == 0:
        return False

    mol = supplier[0]
    if mol is None:
        return False

    try:
        Chem.SanitizeMol(mol, Chem.SanitizeFlags.SANITIZE_ALL)  # Ensures proper structure
        kekule = Chem.MolToSmiles(mol, kekuleSmiles=True)  # Ensure Kekulé representation
        return True
    except:
        return False

# Example usage
ligand_sdf = "/content/1a0q_ligand.sdf"
print("Ligand valid:", validate_ligand_structure(ligand_sdf))


Ligand valid: True


[04:46:39] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


# Include only protein-ligand and ligand-water interactions, ligand chains within 4 Å, and pockets defined by PLIP interactions and neighboring residues

In [ ]:
import numpy as np

def parse_pdb(file_path):
    """
    Parses a PDB file and extracts:
    - Protein atoms
    - Water molecules (HOH)

    Returns:
        tuple: (protein_atoms, water_atoms)
    """
    protein_atoms = []
    water_atoms = []

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith("ATOM") or line.startswith("HETATM"):
                residue_name = line[17:20].strip()  # Residue name (HOH for water)
                atom_type = line[76:78].strip()
                x, y, z = float(line[30:38]), float(line[38:46]), float(line[46:54])
                atom_data = (atom_type, np.array([x, y, z]))

                if residue_name == "HOH":  # Identifying water molecules
                    water_atoms.append(atom_data)
                else:
                    protein_atoms.append(atom_data)

    return protein_atoms, water_atoms

def compute_min_distance(set1, set2):
    """Computes the minimum distance between two sets of atoms."""
    if not set2:  # If the second set is empty, return a large value
        return float('inf')

    min_dist = float('inf')
    for _, coord1 in set1:
        for _, coord2 in set2:
            distance = np.linalg.norm(coord1 - coord2)
            min_dist = min(min_dist, distance)

    return min_dist

def filter_interactions(ligand_atoms, protein_atoms, water_atoms, max_distance=4.0):
    """
    Filters interactions to include only:
    - Protein-ligand interactions
    - Ligand-water interactions
    - Ligand chains within 4 Å

    Returns:
        bool: True if valid interactions exist, False otherwise.
    """
    ligand_protein_dist = compute_min_distance(ligand_atoms, protein_atoms)
    ligand_water_dist = compute_min_distance(ligand_atoms, water_atoms)

    return ligand_protein_dist <= max_distance or ligand_water_dist <= max_distance

# Example usage
ligand_atoms = parse_sdf("/content/1a0q_ligand.sdf")
protein_atoms, water_atoms = parse_pdb("/content/1a0q_protein.pdb")  # Separate water molecules

print("Valid interactions:", filter_interactions(ligand_atoms, protein_atoms, water_atoms))


[04:52:47] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.


Valid interactions: True


# Include entries with good conformations, non-polymeric ligands, non-racemic mixtures, supported elements, and non-highly symmetric ligands.

In [ ]:
from rdkit import Chem
from rdkit.Chem import Descriptors

def validate_ligand_properties(sdf_file):
    """
    Checks if a ligand meets the following criteria:
    - Good 3D conformation
    - Non-polymeric ligand
    - Non-racemic mixture
    - Contains only supported elements
    - Not highly symmetric

    Parameters:
        sdf_file (str): Path to the ligand SDF file.

    Returns:
        bool: True if the ligand is valid, False otherwise.
    """
    supported_elements = {"C", "H", "O", "N", "S", "P", "F", "Cl", "Br", "I"}  # Common organic elements

    supplier = Chem.SDMolSupplier(sdf_file, removeHs=False)
    if not supplier or len(supplier) == 0:
        return False  # Invalid file or empty ligand

    mol = supplier[0]
    if mol is None:
        return False  # Failed to parse ligand

    try:
        # 1. Check if ligand has a valid 3D conformation
        if mol.GetNumConformers() == 0:
            return False  # No 3D structure available

        # 2. Check if ligand is non-polymeric (avoid peptides & nucleotides)
        if Chem.rdMolDescriptors.CalcNumRotatableBonds(mol) > 10:
            return False  # Large flexible ligands are usually polymeric

        # 3. Ensure ligand is non-racemic (chiral center consistency)
        if Chem.rdMolDescriptors.CalcNumAtomStereoCenters(mol) > 1:
            return False  # Likely a racemic mixture

        # 4. Ensure ligand contains only supported elements
        for atom in mol.GetAtoms():
            if atom.GetSymbol() not in supported_elements:
                return False  # Unsupported element found

        # 5. Check for high symmetry (high rotational symmetry score)
        symmetry = Descriptors.FractionCSP3(mol)  # Measures symmetry level
        if symmetry < 0.1:  # Highly planar and symmetrical molecules
            return False

        return True  # Passes all checks

    except Exception as e:
        print(f"Error processing ligand: {e}")
        return False

# Example usage
ligand_sdf = "/content/1a0q_ligand.sdf"
print("Ligand valid:", validate_ligand_properties(ligand_sdf))


Ligand valid: False


[04:56:28] Warning: molecule is tagged as 2D, but at least one Z coordinate is not zero. Marking the mol as 3D.
